# Analysis of Model judgments for robustness to question variation

In [1]:
import numpy as np
import pandas as pd

import output_processing as op

/Users/abhishekp/miniconda3/envs/prej_prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_list = [
    # Main set of models with instruct divide and size variety
    "meta-llama/Llama-3.2-1B",
    "meta-llama/Llama-3.2-1B-Instruct",
    "meta-llama/Llama-3.2-3B",
    "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Llama-3.1-8B",
    "meta-llama/Llama-3.1-8B-Instruct",
    "meta-llama/Llama-3.3-70B-Instruct",
    # Small reference model - would allow for pretraining variation
    "gpt2-medium",
    # Other open models
    "allenai/OLMo-2-1124-7B",
    "allenai/OLMo-2-1124-7B-Instruct",
    "mistralai/Ministral-8B-Instruct-2410",
    "google/gemma-7b",
    "google/gemma-7b-it",
    # Large platform model
    "openai-gpt-4",
]

def read_and_organize_model_results(model_name):
    model_results = pd.read_csv(f"runs/runs-42_07_16/{model_name}-results.csv")
    model_results.replace([0.0], -65504, inplace=True)
    model_results = op.organize_distribution(model_results)
    model_results["model_name"] = model_name.split("/")[-1]
    model_results.loc[model_results["Covered"] == True, "Judgment"] = "Covered"
    model_results.loc[model_results["Covered"] == True, "Judgment_prob"] = model_results["Covered_prob"]
    model_results.loc[model_results["NotCovered"] == True, "Judgment"] = "NotCovered"
    model_results.loc[model_results["NotCovered"] == True, "Judgment_prob"] = model_results["NotCovered_prob"]
    return model_results

In [3]:
# TODO organize the output for better clarity
combined_results = [read_and_organize_model_results(model_name) for model_name in model_list]
combined_df = pd.concat(combined_results).reset_index()
combined_df.loc[:, "item"] = combined_df["title"] + combined_df["version"]
combined_df.shape

(17388, 28)

In [4]:
# Print summary of the experimental results
print(combined_df.model_name.unique())

['Llama-3.2-1B' 'Llama-3.2-1B-Instruct' 'Llama-3.2-3B'
 'Llama-3.2-3B-Instruct' 'Llama-3.1-8B' 'Llama-3.1-8B-Instruct'
 'Llama-3.3-70B-Instruct' 'gpt2-medium' 'OLMo-2-1124-7B'
 'OLMo-2-1124-7B-Instruct' 'Ministral-8B-Instruct-2410' 'gemma-7b'
 'gemma-7b-it' 'openai-gpt-4']


In [5]:
# Looking at one of the result samples
combined_df.sample()

,index,title,prompt_type,prompt,version,output,output_text,cum_logprob,YES_probs,Yes_probs,...,UnAff_prob,Covered_prob,NotCovered_prob,Covered,NotCovered,entropy,model_name,Judgment,Judgment_prob,item
14640,978,Escape of Water I,options,Carol's home insurance policy includes coverag...,unambiguous_uncovered,B,B. Carol is not covered.\nExplanation:\nThe p...,NaN,-9.326406,0.001241,...,0.300771,0.249348,0.300771,False,True,0.895992,gemma-7b,NotCovered,0.300771,Escape of Water Iunambiguous_uncovered


## Prepare the table with Categorical counts and Distributional Spread

In [6]:
# Categorical Counts for Yes/No question variant
question_variant_mask = combined_df["prompt_type"] == "yes_or_no"
yes_or_no_df = combined_df[question_variant_mask]
count_labels = yes_or_no_df.groupby('model_name', as_index=False, sort=False).aggregate(
    {
        'Covered': 'sum',
        'NotCovered': 'sum',
    }
)
count_labels = count_labels.rename({'model_name' : 'Model'}, axis=1)
count_labels

,Model,Covered,NotCovered
0,Llama-3.2-1B,138,0
1,Llama-3.2-1B-Instruct,138,0
2,Llama-3.2-3B,129,9
3,Llama-3.2-3B-Instruct,51,87
4,Llama-3.1-8B,75,63
5,Llama-3.1-8B-Instruct,0,138
6,Llama-3.3-70B-Instruct,59,79
7,gpt2-medium,5,133
8,OLMo-2-1124-7B,73,65
9,OLMo-2-1124-7B-Instruct,53,85


In [7]:
# Latex table just for categorical counts
count_labels.to_latex("reports/yes_or_no_categorical_counts.tex", index=True, float_format="%.2f")

In [8]:
# Distributional spread for Yes/No question variant
question_variant_mask = combined_df["prompt_type"] == "yes_or_no"
yes_or_no_df = combined_df[question_variant_mask]

# Util function
yes_or_no_judgement_range = yes_or_no_df[['model_name', 'Covered_prob', 'NotCovered_prob']].melt(id_vars='model_name', value_vars=['Covered_prob', 'NotCovered_prob']).groupby('model_name', as_index=False).agg(
    Min = pd.NamedAgg('value', lambda x: np.round(np.min(x), 2)),
    Max = pd.NamedAgg('value', lambda x: np.round(np.max(x), 2)),
    ).rename({"model_name": "Model"}, axis=1)

yes_or_no_judgement_range

,Model,Min,Max
0,Llama-3.1-8B,0.12,0.34
1,Llama-3.1-8B-Instruct,0.09,0.70
2,Llama-3.2-1B,0.06,0.27
3,Llama-3.2-1B-Instruct,0.13,0.50
4,Llama-3.2-3B,0.08,0.50
5,Llama-3.2-3B-Instruct,0.15,0.65
6,Llama-3.3-70B-Instruct,0.02,0.81
7,Ministral-8B-Instruct-2410,0.20,0.58
8,OLMo-2-1124-7B,0.18,0.56
9,OLMo-2-1124-7B-Instruct,0.00,0.99


In [9]:
# Separate latex table for yes_or_no judgement range
yes_or_no_judgement_range.to_latex("reports/yes_or_no_distributional_spread.tex", index=True, float_format="%.2f")

In [10]:
# Combined table
# Merge the two tables
yes_or_no_table = pd.concat([count_labels.set_index('Model'), yes_or_no_judgement_range.set_index('Model')], axis=1, join='inner')
yes_or_no_table

,Covered,NotCovered,Min,Max
Model,,,,
Llama-3.2-1B,138,0,0.06,0.27
Llama-3.2-1B-Instruct,138,0,0.13,0.50
Llama-3.2-3B,129,9,0.08,0.50
Llama-3.2-3B-Instruct,51,87,0.15,0.65
Llama-3.1-8B,75,63,0.12,0.34
Llama-3.1-8B-Instruct,0,138,0.09,0.70
Llama-3.3-70B-Instruct,59,79,0.02,0.81
gpt2-medium,5,133,0.12,0.30
OLMo-2-1124-7B,73,65,0.18,0.56


In [11]:
yes_or_no_table.to_latex("reports/yes_or_no_table.tex", index=True, float_format="%.2f")

## Prepare the majority votes frequency table

In [12]:
# Calculate the majority votes
majority_vote_by_model = combined_df.groupby(['title', 'version', 'model_name'], as_index=False, sort=False).aggregate(
     {
        'Covered': 'sum',
        'NotCovered': 'sum',
    }
)
majority_vote_by_model.loc[:, "majority_count"] = majority_vote_by_model[['Covered', 'NotCovered']].max(axis=1)
majority_vote_by_model.shape
majority_vote_table_df = majority_vote_by_model[["model_name", "majority_count"]].value_counts().reset_index(name="Frequency")\
.pivot_table(columns =['majority_count'], index="model_name", aggfunc="sum", margins=True)\

majority_vote_table_df.replace(np.nan, 0, inplace=True)
# Remove the "All" from columns and rows
majority_vote_table_df = majority_vote_table_df.drop(["All"], axis=0)
majority_vote_table_df = majority_vote_table_df.drop([("Frequency", "All")], axis=1)

majority_vote_table_df = majority_vote_table_df.rename({'model_name': 'Model', 'majority_count': 'Majority Count'})
majority_vote_table_df.index = majority_vote_table_df.index.rename('Model')
majority_vote_table_df.columns.names = [None, 'Majority Count']
majority_vote_table_df

Frequency                       
Majority Count                     5     6     7     8    9
Model                                                      
Llama-3.1-8B                    40.0  50.0  48.0   0.0  0.0
Llama-3.1-8B-Instruct            8.0  41.0  56.0  29.0  4.0
Llama-3.2-1B                    10.0  59.0  69.0   0.0  0.0
Llama-3.2-1B-Instruct          129.0   9.0   0.0   0.0  0.0
Llama-3.2-3B                    79.0  55.0   4.0   0.0  0.0
Llama-3.2-3B-Instruct           77.0  46.0  15.0   0.0  0.0
Llama-3.3-70B-Instruct          24.0  34.0  78.0   2.0  0.0
Ministral-8B-Instruct-2410      20.0  67.0  26.0  24.0  1.0
OLMo-2-1124-7B                  47.0  64.0  20.0   7.0  0.0
OLMo-2-1124-7B-Instruct         51.0  57.0  30.0   0.0  0.0
gemma-7b                        44.0  60.0  29.0   5.0  0.0
gemma-7b-it                     10.0  33.0  79.0  16.0  0.0
gpt2-medium                     50.0  83.0   5.0   0.0  0.0
openai-gpt-4                    61.0  45.0  27.0   5.0  0.0

In [13]:
majority_vote_table_df.to_latex("reports/majority-votes-freq-table.tex", float_format="%.0f")

## Minority responses

In [14]:
# Minority responses
combined_df.loc[combined_df.Covered == True, "judgment"] = "Covered"
combined_df.loc[combined_df.NotCovered == True, "judgment"] = "Not Covered"
majority_vote_by_model.loc[:, "majority"] = majority_vote_by_model.apply(lambda x: "Covered" if x.Covered >= x.NotCovered else "Not Covered",  axis=1)
combined_df.loc[:, "in_minority"]= False
for index, row in majority_vote_by_model.iterrows():
    item_model_mask = (combined_df["title"] == row.title) & (combined_df["version"] == row.version) & (combined_df["model_name"] == row.model_name)
    in_minority_mask = combined_df["judgment"]!= row.majority
    combined_df.loc[item_model_mask & in_minority_mask, "in_minority"] = True

combined_df.in_minority.sum(), combined_df.shape[0]

## Minority responses by question variants for each model

(5769, 17388)

In [15]:
minority_responses = combined_df[combined_df.in_minority == True]

In [16]:
minority_pivot = minority_responses[["model_name", "prompt_type"]].value_counts().reset_index().pivot_table(index="model_name", columns="prompt_type")
minority_pivot.replace(np.nan, 0, inplace=True)
minority_pivot

count                                  \
prompt_type                agreement agreement_negation disagreement   
model_name                                                             
Llama-3.1-8B                    21.0              101.0        110.0   
Llama-3.1-8B-Instruct            5.0               33.0         58.0   
Llama-3.2-1B                     0.0              138.0        138.0   
Llama-3.2-1B-Instruct           12.0              126.0        126.0   
Llama-3.2-3B                   106.0              138.0        108.0   
Llama-3.2-3B-Instruct           51.0               70.0         70.0   
Llama-3.3-70B-Instruct          14.0               57.0         46.0   
Ministral-8B-Instruct-2410       8.0               52.0         80.0   
OLMo-2-1124-7B                  62.0               57.0         67.0   
OLMo-2-1124-7B-Instruct         34.0               68.0        106.0   
gemma-7b                        22.0               63.0         95.0   
gemma-7b-it                      8.0                5.0        127.0   
gpt2-medium                    125.0               13.0         13.0   
openai-gpt-4                    32.0               56.0         41.0   

                                                                             \
prompt_type                disagreement_negation negation no_or_yes options   
model_name                                                                    
Llama-3.1-8B                                36.0     31.0      15.0    30.0   
Llama-3.1-8B-Instruct                       75.0     33.0       1.0     1.0   
Llama-3.2-1B                                 0.0      0.0      14.0    48.0   
Llama-3.2-1B-Instruct                       12.0      0.0     126.0   117.0   
Llama-3.2-3B                                 0.0      0.0       0.0   128.0   
Llama-3.2-3B-Instruct                       68.0     40.0      32.0    61.0   
Llama-3.3-70B-Instruct                      89.0     99.0      10.0    19.0   
Ministral-8B-Instruct-2410                  59.0     51.0       5.0    80.0   
OLMo-2-1124-7B                              67.0     29.0      27.0    46.0   
OLMo-2-1124-7B-Instruct                     70.0     48.0       9.0    26.0   
gemma-7b                                    47.0     30.0      14.0    71.0   
gemma-7b-it                                104.0     26.0       5.0     5.0   
gpt2-medium                                125.0     42.0       4.0    91.0   
openai-gpt-4                                72.0     44.0      58.0    56.0   

                                                      
prompt_type                options_flipped yes_or_no  
model_name                                            
Llama-3.1-8B                          31.0      31.0  
Llama-3.1-8B-Instruct                 89.0       1.0  
Llama-3.2-1B                          17.0       0.0  
Llama-3.2-1B-Instruct                 12.0      12.0  
Llama-3.2-3B                           0.0       9.0  
Llama-3.2-3B-Instruct                 61.0      23.0  
Llama-3.3-70B-Instruct                18.0       4.0  
Ministral-8B-Instruct-2410            17.0       5.0  
OLMo-2-1124-7B                        60.0      12.0  
OLMo-2-1124-7B-Instruct               57.0      17.0  
gemma-7b                              34.0      43.0  
gemma-7b-it                           31.0       2.0  
gpt2-medium                           38.0       8.0  
openai-gpt-4                          54.0      25.0

## Minority responses by question variants

In [17]:
# Minority responses
combined_df.loc[combined_df.Covered == True, "judgment"] = "Covered"
combined_df.loc[combined_df.NotCovered == True, "judgment"] = "Not Covered"
majority_vote_by_model.loc[:, "majority"] = majority_vote_by_model.apply(lambda x: "Covered" if x.Covered >= x.NotCovered else "Not Covered",  axis=1)
combined_df.loc[:, "in_minority"]= False
for index, row in majority_vote_by_model.iterrows():
    item_model_mask = (combined_df["title"] == row.title) & (combined_df["version"] == row.version) & (combined_df["model_name"] == row.model_name)
    in_minority_mask = combined_df["judgment"]!= row.majority
    combined_df.loc[item_model_mask & in_minority_mask, "in_minority"] = True

combined_df.in_minority.sum(), combined_df.shape[0]

(5769, 17388)

In [18]:
minority_responses = combined_df[combined_df.in_minority == True]
minority_responses[["prompt_type"]].value_counts().reset_index()

,prompt_type,count
0,disagreement,1185
1,agreement_negation,977
2,disagreement_negation,824
3,options,779
4,options_flipped,519
5,agreement,500
6,negation,473
7,no_or_yes,320
8,yes_or_no,192


In [19]:
(minority_responses[["prompt_type"]].value_counts()/minority_responses.shape[0]).reset_index()

,prompt_type,count
0,disagreement,0.205408
1,agreement_negation,0.169353
2,disagreement_negation,0.142832
3,options,0.135032
4,options_flipped,0.089964
5,agreement,0.086670
6,negation,0.081990
7,no_or_yes,0.055469
8,yes_or_no,0.033281


In [20]:
minority_counts = minority_responses[["prompt_type"]].value_counts().reset_index().set_index("prompt_type")
minority_counts.columns = ['Count']
minority_props = (minority_counts/minority_responses.shape[0]).reset_index().set_index("prompt_type")
minority_props.columns = ['Proportion']
pd.concat([minority_counts, minority_props], axis=1, join='inner')

,Count,Proportion
prompt_type,,
disagreement,1185,0.205408
agreement_negation,977,0.169353
disagreement_negation,824,0.142832
options,779,0.135032
options_flipped,519,0.089964
agreement,500,0.086670
negation,473,0.081990
no_or_yes,320,0.055469
yes_or_no,192,0.033281


In [21]:
pd.concat([minority_counts, minority_props], axis=1, join='inner').columns

Index(['Count', 'Proportion'], dtype='object')

## Make the table for Jensen-Shannon Distance for non Yes/No question variant

In [22]:
divergences = op.calculate_relative_measures(combined_df)
divergences.sample()

,title,version,model_name,prompt_type,js_dist,kl_div
12100,Vehicle Glass I,unambiguous_uncovered,Ministral-8B-Instruct-2410,disagreement,0.062967,0.016228


In [23]:
max_distance_variant_for_model = divergences.groupby(['model_name', 'prompt_type'], as_index=False, sort=False).aggregate(
    Mean = pd.NamedAgg('js_dist', "mean"),
    Std = pd.NamedAgg('js_dist', "std")
).groupby(['model_name'], as_index=False, sort=False).apply(lambda x: x.loc[x["Mean"].idxmax(), :])
max_distance_variant_for_model = max_distance_variant_for_model.rename({'model_name' : 'Model', 'prompt_type': 'Variant'}, axis=1)
max_distance_variant_for_model = max_distance_variant_for_model.replace(
    {
        "agreement_negation": "Agr. w/ Neg.", 
        "no_or_yes": "N/Y", 
        "disagreement_negation": "Disagr. w/ Neg.", 
        "options_flipped": "Options F.", 
        "options": "Options", 
        "negation": "Negation",
    }
)
max_distance_variant_for_model

/var/folders/0m/231b9j890r542zfr6djkqlz40000gp/T/ipykernel_9819/4163965735.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).groupby(['model_name'], as_index=False, sort=False).apply(lambda x: x.loc[x["Mean"].idxmax(), :])


,Model,Variant,Mean,Std
0,Llama-3.2-1B,Agr. w/ Neg.,0.234301,0.033502
1,Llama-3.2-1B-Instruct,N/Y,0.244847,0.031106
2,Llama-3.2-3B,Agr. w/ Neg.,0.185209,0.055686
3,Llama-3.2-3B-Instruct,Options F.,0.260064,0.050909
4,Llama-3.1-8B,Options F.,0.095668,0.051488
5,Llama-3.1-8B-Instruct,Options F.,0.277773,0.027844
6,Llama-3.3-70B-Instruct,Negation,0.329654,0.167655
7,gpt2-medium,Disagr. w/ Neg.,0.166143,0.019285
8,OLMo-2-1124-7B,Disagr. w/ Neg.,0.382940,0.052281
9,OLMo-2-1124-7B-Instruct,Options F.,0.532485,0.115830


In [24]:
max_distance_variant_for_model.to_latex("reports/robustness-prompt-type-distance.tex", float_format="%0.2f", index=False)